# Ordering and Delivery
DLM edited
This notebook demonstrates ordering and download with the orders api. In this notebook, we check authentication by requesting an orders list. We poll for order success then download images individually. And finally, we create, poll, and download the same order delivered as a single zip file.

Modified from Amy Dixon's Ordering and Delivery code

In [1]:
import json
import os
import pathlib
import time
import re

import requests
from requests.auth import HTTPBasicAuth

## Authenticating and setup

In [3]:
# API Key stored as an env variable
# Paste Personal API key here
PLANET_API_KEY = "PLAKbd4eb24d0cbc4c8887f146944145df01"

In [1]:
# SET INITIAL parameters

order_name = "2023 May 8b 2"
output_dir = '/Volumes/NYC_geo/Planet/raw_images/time_series' #'/Users/dlm356/dlm356_files/nyc_trees/planet_output_test_dir'

data_product = 'analytic_8b_sr_udm2'
# options are 'analytic_sr_udm2' for 4band
# or 'analytic_8b_sr_udm2' for 8band (SuperDove, if available)
# future: any way to setup to look for both and prioritize 8 band?
# do a 4 band search AND an 8 band search, and wipe out any 4 band images that are the same acquisition as the 8 band? Then do two submission requests?

# Time range
date_gte = "2023-05-01T00:00:00Z" # date and time to start
date_lte = "2023-06-01T00:00:00Z" # date and time to end

# Filters
max_cloud = 0.1 # maximum cloud cover
min_cloud = 0.0 # if you want a min cloud filter, get all
max_view_angle = 3.0 # maximum view angle

# Set geometry
# Pull from json file in lon lat format, can be a polygon that does not need to be a bounding box
# convex hull for NYC, from nyc_buffer_dissolved_filled_convex_hull_latlon.geojson.
geometry = {
  "type":"Polygon",
  "coordinates": [ [ [ -74.246842270352175, 40.487128599923757 ], [ -74.247047482532864, 40.487129667687171 ], [ -74.247557691590231, 40.487140753475323 ], [ -74.247762651227603, 40.487148597897679 ], [ -74.24827175482362, 40.487176524273345 ], [ -74.248476078291645, 40.487191130672088 ], [ -74.248983123527367, 40.487235845364943 ], [ -74.249186428388754, 40.487257186400988 ], [ -74.249690466211277, 40.487318605717469 ], [ -74.249892371932077, 40.487346641446301 ], [ -74.250392456422787, 40.487424650429126 ], [ -74.250592583813059, 40.487459328376808 ], [ -74.251087780747682, 40.4875537810114 ], [ -74.25128575649174, 40.487595036268232 ], [ -74.251775139645105, 40.487705755762292 ], [ -74.251970593166931, 40.487753511108629 ], [ -74.252453246489708, 40.487880290221874 ], [ -74.252645811926598, 40.487934456270835 ], [ -74.253120831945807, 40.488077057701489 ], [ -74.253310148832185, 40.488137533066151 ], [ -74.253776646339873, 40.488295689894379 ], [ -74.253962360281179, 40.488362361377028 ], [ -74.254419461996889, 40.488535777563818 ], [ -74.254601225332081, 40.488608520367748 ], [ -74.255048075535427, 40.488796871308075 ], [ -74.255225547987067, 40.488875549270603 ], [ -74.255661310117972, 40.489078482400259 ], [ -74.255834159428545, 40.489162948247383 ], [ -74.256258017651817, 40.489380083701882 ], [ -74.256425920205174, 40.489470179323462 ], [ -74.257221960924113, 40.48993719273119 ], [ -74.257296940563336, 40.489985189813481 ], [ -74.258894235230372, 40.491227161564503 ], [ -74.262031420729414, 40.494156658161366 ], [ -74.262440346294667, 40.494550056071184 ], [ -74.262525178591389, 40.494636512053106 ], [ -74.262904747329131, 40.495046700379653 ], [ -74.262983135715132, 40.495136632028235 ], [ -74.263332166755148, 40.495562335362287 ], [ -74.263403867343513, 40.495655463047655 ], [ -74.263721274690823, 40.496095357708697 ], [ -74.263786064373335, 40.496191391856598 ], [ -74.26407086029144, 40.496644110005562 ], [ -74.264128537433891, 40.496742751997132 ], [ -74.264379835523371, 40.497206885886747 ], [ -74.26565659985944, 40.49958682805272 ], [ -74.266170540379122, 40.500556148675187 ], [ -74.266575986735319, 40.501454408656187 ], [ -74.266621387470622, 40.501575366633141 ], [ -74.266788448659909, 40.502073812209424 ], [ -74.266824726673278, 40.502196551984014 ], [ -74.266954226964302, 40.502701466975765 ], [ -74.266981263593621, 40.502825587947797 ], [ -74.267072779250469, 40.503335324381631 ], [ -74.267090485987438, 40.503460421431519 ], [ -74.267143717221813, 40.503973315550702 ], [ -74.26738434297279, 40.507420581633731 ], [ -74.267386557773577, 40.507488212136423 ], [ -74.266900255348475, 40.510281485831442 ], [ -74.258236133079492, 40.547625051036107 ], [ -74.211407740090849, 40.634745313500396 ], [ -74.209849660040163, 40.636870012230148 ], [ -74.204301122444122, 40.642562465468814 ], [ -73.920366778038897, 40.920012235631425 ], [ -73.919743818478253, 40.920573071711942 ], [ -73.919672360321172, 40.920632327778293 ], [ -73.916877519423153, 40.922340161144973 ], [ -73.916706157811731, 40.922415689805959 ], [ -73.906428917121644, 40.923958714813352 ], [ -73.84831360516101, 40.919134562731657 ], [ -73.847937182420523, 40.919056399863592 ], [ -73.778727373519246, 40.888520922748214 ], [ -73.76867977058113, 40.882338192425401 ], [ -73.763703088645158, 40.878815235250023 ], [ -73.75696036920688, 40.872481074144403 ], [ -73.689886430134905, 40.753627565703233 ], [ -73.689875105703436, 40.753559752636193 ], [ -73.689600416993756, 40.751914720070779 ], [ -73.689535948275505, 40.751419440491432 ], [ -73.689102289709453, 40.747880991951732 ], [ -73.688249028357561, 40.740361661254077 ], [ -73.688226127131912, 40.738447694662021 ], [ -73.688237283629448, 40.738360824370822 ], [ -73.726990665989518, 40.592217349698473 ], [ -73.727022447257568, 40.592123563668316 ], [ -73.727954535813126, 40.59025777301418 ], [ -73.72796603631393, 40.590240723071105 ], [ -73.732674633834478, 40.586512086409762 ], [ -73.732690475197373, 40.58650528436074 ], [ -73.733548873915751, 40.586268154266889 ], [ -73.938289442679434, 40.533022837810584 ], [ -73.938467100888147, 40.532996292014708 ], [ -74.246842270352175, 40.487128599923757 ] ] ]
}

In [4]:
orders_url = 'https://api.planet.com/compute/ops/orders/v2'
data_url = "https://api.planet.com/data/v1"

### Curl example

To check your orders list and make sure you have the permissions you need, uncomment the following line to run `curl`

In [5]:
!curl -L -H "Authorization: api-key $PLANET_API_KEY" $orders_url
!curl -u PLANET_API_KEY: https://api.planet.com/data/v1/ # DLM: authentication of API key if needed, https://developers.planet.com/docs/apis/data/api-mechanics/

curl: (2) no URL specified
curl: try 'curl --help' for more information
{"_links": {"_self": "https://api.planet.com/data/v1/", "asset-types": "https://api.planet.com/data/v1/asset-types/", "item-types": "https://api.planet.com/data/v1/item-types/", "spec": "https://api.planet.com/data/v1/spec"}}

## Searching with the Data API
We can use the [data API](https://developers.planet.com/docs/apis/data/) in order to automate searching based on the search criterias like: date range, cloud cover, area cover, aoi.

In [6]:
# Setup filters
# get images that overlap with our AOI 
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geometry
}

# get images acquired within a date range
# change this here!
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": date_gte, # "2021-07-20T00:00:00Z", #"gte":"2024-03-18T00:00:00Z", #gte greater than or equal to
    "lte": date_lte # "2021-07-24T00:00:00Z", #"lte":"2024-04-17T23:00:00Z"
  }
}

# only get images which have <10% cloud coverage # DLM: might need to change this tolerance for NYC to get more coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "gte": min_cloud, # if we want a min cloud filter to get more data
    "lte": max_cloud # 0.1
  }
}

# only images with <3 degree view angle
view_angle_filter = {
    "type": "RangeFilter",
    "field_name": "view_angle",
    "config": {
        "lte": max_view_angle # 3.0
    }
}

# combine our geo, date, cloud filters
combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter, view_angle_filter]
}

In [7]:
combined_filter

{'type': 'AndFilter',
 'config': [{'type': 'GeometryFilter',
   'field_name': 'geometry',
   'config': {'type': 'Polygon',
    'coordinates': [[[-74.24684227035218, 40.48712859992376],
      [-74.24704748253286, 40.48712966768717],
      [-74.24755769159023, 40.48714075347532],
      [-74.2477626512276, 40.48714859789768],
      [-74.24827175482362, 40.487176524273345],
      [-74.24847607829165, 40.48719113067209],
      [-74.24898312352737, 40.48723584536494],
      [-74.24918642838875, 40.48725718640099],
      [-74.24969046621128, 40.48731860571747],
      [-74.24989237193208, 40.4873466414463],
      [-74.25039245642279, 40.487424650429126],
      [-74.25059258381306, 40.48745932837681],
      [-74.25108778074768, 40.4875537810114],
      [-74.25128575649174, 40.48759503626823],
      [-74.2517751396451, 40.48770575576229],
      [-74.25197059316693, 40.48775351110863],
      [-74.25245324648971, 40.487880290221874],
      [-74.2526458119266, 40.487934456270835],
      [-74.253120

In [8]:
# Search for available scenes based on filter parameters above
item_type = "PSScene"

# API request object
search_request = {
  "item_types": [item_type], 
  "filter": combined_filter
}

# fire off the POST request
search_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(PLANET_API_KEY, ''),
    json=search_request)

# extract image IDs only
image_ids = [feature['id'] for feature in search_result.json()['features']]
print(image_ids)
# These are the image ids that are needed to submit an order, might be need to be looped and filtered

if len(image_ids) == 250:
  print('WARNING: 250 search items reached, search result will be limited to 250. Pagination required for larger geojson search')
else:
  print(str(len(image_ids)) + " image items")

NameError: name 'requests' is not defined

### Requests example

In this notebook, we will be using `requests` to communicate with the orders v2 API. First, we will check our orders list to make sure authentication and communication is working as expected.

We want to get a response code of `200` from this API call. To troubleshoot other response codes, see the [List Orders](https://developers.planet.com/docs/orders/reference/#operation/listOrders) AOI reference.

In [15]:
# check authorization, want <200> response if all permissions are good
auth = HTTPBasicAuth(PLANET_API_KEY, '')
response = requests.get(orders_url, auth=auth)
response

<Response [200]>

## Ordering

In this example, we will order two `PSScene4Band` analytic images. For variations on this kind of order, see [Ordering Data](https://developers.planet.com/docs/orders/ordering-delivery/#ordering-data_1).

In this order, we request an `analytic` bundle. A bundle is a group of assets for an item. The `analytic` bundle for the  `PSScene4Band` item contains 3 assets: the analytic image, the analytic xml file, and the udm. See the [Product bundles reference](https://developers.planet.com/docs/orders/product-bundles-reference/) to learn about other bundles and other items.

Now we will list the names of orders we have created thus far. Your list may be empty if you have not created an order yet.

In [16]:
# check previous orders (if necessary)
orders = response.json()['orders']
[r['name'] for r in orders]

['HZTX 2024 Aug 4b',
 'HZTX 2024 July 4b',
 'HZTX 2024 June 4b',
 'HZTX 2024 May 4b',
 'HZTX 2024 Apr 4b',
 'HZTX 2024 Mar 4b',
 'HZTX 2024 Feb 4b',
 'HZTX 2024 Jan 4b',
 'HZTX 2024 Jan 4b',
 '2019 Dec 4b cloud gte 10',
 '2019 Nov 4b cloud gte 10',
 '2019 Oct 4b cloud gte 10',
 '2019 Sep 4b cloud gte 10',
 '2019 Aug 4b cloud gte 10',
 '2019 July 4b cloud gte 10',
 '2019 June 4b cloud gte 10',
 '2019 May 4b cloud gte 10',
 '2019 April 4b cloud gte 10',
 '2019 March 4b cloud gte 10',
 '2019 Feb 4b cloud gte 10']

### Place Order

In [17]:
# set content type to json
headers = {'content-type': 'application/json'}

In [18]:
# This is where to indicate exactly which images you want to download, but you can add clipping or bandmath below annd order from there
product = [
    {
      "item_ids": image_ids, #list of image ids
      "item_type":  "PSScene", #indicate the item type, PSScene is PlanetScope
      "product_bundle": data_product
    }
]

# define the clip tool with geometry from above
clip = {
    "clip": {
        "aoi": geometry
    }
}

# DLM: harmonize to sentinel-2
# https://support.planet.com/hc/en-us/articles/4405971577501-How-Why-and-When-to-Use-the-New-Target-Sensor-in-the-Harmonization-Tool
harmonize = {
    "harmonize": {
        "target_sensor": "Sentinel-2"
      }
}

tool_request = { 
    "name": order_name, #"toolchain order harmonized 8b", # need to change name for each order or set up something procedural to be updated in the script
    "products": product,
    "tools": [clip, harmonize],
    "delivery": {"single_archive": True, "archive_type": "zip"}
}

In [19]:
# parse error in json response to filter out images that aren't actually available
def parse_response(data):
    
    results = []
    for detail in data['field']['Details']:
        message = detail['message']
        parts = message.split('/')[1].split('[')[0]  # Extract the substring between '/' and '['
        results.append(parts)
        
    return(results)

# DLM: function for placing an order and screening for bad image links before submitting
def place_order_filtered(request, auth):
    response = requests.post(orders_url, data=json.dumps(request), auth=auth, headers=headers)
    print(response.json())

    bad_ids = parse_response(response.json()) # TO FIX: throws an error if there are no bad ids (but submits the order anyway?)
    image_ids_filtered = [entry for entry in image_ids if entry not in bad_ids]

    print(bad_ids)
    print(image_ids)
    print(image_ids_filtered)

    # revise product and request based on the filtered ids
    product = [
        {
        "item_ids": image_ids_filtered, #list of image ids
        "item_type":  "PSScene", #indicate the item type, PSScene is PlanetScope
        "product_bundle": data_product
        }
    ]

    request = { 
        "name": order_name, # "toolchain order harmonized filter test", # need to change name for each order or set up something procedural to be updated in the script
        "products": product,
        "tools": [clip, harmonize], 
        "delivery": {"single_archive": True, "archive_type": "zip"}
    } # this is the tool request

    # try again with the filtered data
    response = requests.post(orders_url, data=json.dumps(request), auth=auth, headers=headers)
    print(response.json())
    order_id = response.json()['id']
    print(order_id)
    order_url = orders_url + '/' + order_id
    return order_url


In [20]:
# This is the actual order submission step!
tool_order_url = place_order_filtered(tool_request, auth)

{'field': {'Details': [{'message': 'No access to assets: PSScene/20221212_144224_12_242b/[ortho_analytic_4b_sr ortho_analytic_4b_xml ortho_udm2]'}, {'message': 'No access to assets: PSScene/20221227_144357_58_2449/[ortho_analytic_4b_sr ortho_analytic_4b_xml ortho_udm2]'}, {'message': 'No access to assets: PSScene/20221227_144355_27_2449/[ortho_analytic_4b_sr ortho_analytic_4b_xml ortho_udm2]'}, {'message': 'No access to assets: PSScene/20221227_144352_97_2449/[ortho_analytic_4b_sr ortho_analytic_4b_xml ortho_udm2]'}, {'message': 'No access to assets: PSScene/20221227_144350_66_2449/[ortho_analytic_4b_sr ortho_analytic_4b_xml ortho_udm2]'}, {'message': 'No access to assets: PSScene/20221224_144621_55_241e/[ortho_analytic_4b_sr ortho_analytic_4b_xml ortho_udm2]'}, {'message': 'No access to assets: PSScene/20221224_144619_22_241e/[ortho_analytic_4b_sr ortho_analytic_4b_xml ortho_udm2]'}, {'message': 'No access to assets: PSScene/20221219_152200_17_2481/[ortho_analytic_4b_sr ortho_analytic

KeyError: 'id'

### Poll for Order Success

In [45]:
def poll_for_success(order_url, auth, num_loops=5):
    count = 0
    while(count < num_loops):
        count += 1
        r = requests.get(order_url, auth=auth)
        response = r.json()
        state = response['state']
        print(state)
        end_states = ['success', 'failed', 'partial']
        if state in end_states:
            break
        time.sleep(10)
        
poll_for_success(tool_order_url, auth)

success


### View Results
Now lets review our previous order and download it

In [46]:
requests.get(tool_order_url, auth=auth).json()['state']

'success'

In [47]:
r = requests.get(tool_order_url, auth=auth)
response = r.json()
results = response['_links']['results']

In [48]:
[r['name'] for r in results]

['0be91841-cc42-4e93-b440-665deb97157d/output.zip',
 '0be91841-cc42-4e93-b440-665deb97157d/manifest.json']

## Download

### Downloading each asset individually

In [49]:
# downloading assets
def download_results(results, overwrite=False):
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name in zip(results_urls, results_names):
        #path = pathlib.Path(os.path.join('data', name))
        path = pathlib.Path(os.path.join(output_dir, name)) #'/Users/dlm356/dlm356_files/nyc_trees/planet_output_test_dir'
        
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))

In [50]:
download_results(results)

2 items to download
downloading 0be91841-cc42-4e93-b440-665deb97157d/output.zip to /Users/dlm356/dlm356_files/nyc_trees/planet_output_test_dir/0be91841-cc42-4e93-b440-665deb97157d/output.zip
downloading 0be91841-cc42-4e93-b440-665deb97157d/manifest.json to /Users/dlm356/dlm356_files/nyc_trees/planet_output_test_dir/0be91841-cc42-4e93-b440-665deb97157d/manifest.json
